## Jupyter notebook for calculating Bayesian Information Criteria (BIC) for ATAC-seq data

This notebook is the code for calculating BIC for ATAC-seq data used in the paper **"Analysis of chromatin organinization and gene expression in T cells identifies functional genes for rheumatoid arthritis"** by *Jing Yang, Amanda McGovern, Paul Martin, Kate Duffus, Xiangyu Ge, Peyman Zarrineh, Andrew P Morris, Antony Adamson, Peter Fraser, Magnus Rattray & Stephen Eyre*.

Author : *Jing Yang*  <br />
Date: 01/05/2020 <br />
For any questions about the code, please drop me a line at Jing.Yang@manchester.ac.uk

### loading gptk package for Gaussian process regression

In [4]:
library(gptk)
library(tidyverse)
library(gridExtra)

Loading required package: Matrix

Loading required package: fields

Loading required package: spam

Loading required package: dotCall64

Loading required package: grid

Spam version 2.5-1 (2019-12-12) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.


Attaching package: ‘spam’


The following object is masked from ‘package:Matrix’:

    det


The following objects are masked from ‘package:base’:

    backsolve, forwardsolve


Loading required package: maps

See https://github.com/NCAR/Fields for
 an extensive vignette, other supplements and source code 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.1     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ──────────────────────

### read ATACseq data

In [5]:
data <- read.table('data/gene_normalized_new.csv',sep=',',header=T)

In [6]:
head(data)

,ENSG,chr,ENSGStart,ENSGEnd,T01,T201,T601,T2H1,T4H1,T24H1,T02,T202,T602,T2H2,T4H2,T24H2
,<fct>,<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000000003,chrX,100627109,100639991,5.041981,5.062369,4.178733,3.377680,1.986341,5.324410,5.177855,3.688929,3.295454,5.253633,4.226941,4.335357
2,ENSG00000000419,chr20,50934867,50958555,10.556373,11.059740,10.784106,10.332645,10.273519,10.419062,11.032946,11.032608,10.768947,10.781390,10.590316,10.778582
3,ENSG00000000457,chr1,169849631,169894267,8.074191,7.582473,9.025557,9.324986,9.353563,9.026321,7.985902,8.162494,9.417072,9.456439,9.271116,8.889596
4,ENSG00000000460,chr1,169662007,169854080,9.029226,8.655751,9.993021,12.866744,14.215491,10.871978,8.287388,8.345116,10.056483,11.656388,13.905748,11.378513
5,ENSG00000000938,chr1,27612064,27635277,9.680082,9.113586,9.108392,9.024590,9.395413,7.303896,5.479472,5.918468,5.355478,4.750553,5.187896,2.535247
6,ENSG00000000971,chr1,196651878,196747504,6.360314,5.424491,5.394533,5.269632,4.710372,4.385639,5.551854,5.941924,5.636894,5.042941,4.976733,4.017008


In [7]:
idx_rep1 <- paste('T', c('0','20','60','2H','4H','24H'), 1, sep='')
idx_rep2 <- paste('T', c('0','20','60','2H','4H','24H'), 2, sep='')


In [9]:
idx_rep2

[1] "T02"   "T202"  "T602"  "T2H2"  "T4H2"  "T24H2"

### normalized ATACseq data

In [10]:
normalized_rep1 <- t(scale(t(data[,idx_rep1]), center=T, scale=T))
normalized_rep2 <- t(scale(t(data[,idx_rep2]), center=T, scale=T))
normalized_data <- cbind(normalized_rep1, normalized_rep2)

In [11]:
head(normalized_data)

T01,T201,T601,T2H1,T4H1,T24H1,T02,T202,T602,T2H2,T4H2,T24H2
0.68367203,0.6995106,0.01306173,-0.60923250,-1.6900877,0.90307589,1.0834984,-0.8185590,-1.3212106,1.1803018,-0.1312644,0.007233663
-0.04829461,1.6242392,0.70839053,-0.79167122,-0.9881287,-0.50453517,1.1733125,1.1713533,-0.3589981,-0.2867780,-1.3958124,-0.303077301
-0.89992044,-1.5734576,0.40322396,0.81337007,0.8525137,0.40427030,-1.3589754,-1.0856029,0.8565337,0.9174757,0.6305888,0.039980104
-0.86773867,-1.0374602,-0.42975359,0.87617612,1.4890983,-0.03032199,-1.0717937,-1.0450963,-0.2536433,0.4862613,1.5265188,0.357753167
0.88767224,0.2103451,0.20413462,0.10393748,0.5473103,-1.95339972,0.5043608,0.8683552,0.4015517,-0.1000224,0.2626005,-1.936845909
1.61811956,0.2450243,0.20106705,0.01780579,-0.8027747,-1.27924194,0.5224002,1.0927187,0.6467356,-0.2216801,-0.3184822,-1.721692187


### use logscaled time points for GP regression 

In [14]:
time0 <- log(c(0,20,60,120,240,1440)+10)
times_data <- c(time0, time0)

In [19]:
x <- matrix(times_data)
lld_rbf <- numeric(0)
lld_static <- numeric(0)

for (ii in (1:dim(normalized_data)[1])) {  ## this calculation will take a long time
  y <- matrix(normalized_data[ii,])
  model0 <- list() ## Allocate space for model.
  options=gpOptions(approx="ftc")
  options$kern = list(type="cmpnd",comp=list(list(type="rbf"),list(type="white"))) ### use rbf+white nosie kernel for model 0
  ## Optimise GP log likelihoods.
  model0 <- gpCreate(dim(x)[2], dim(y)[2], x, y, options)
  model0 <- gpOptimise(model0,0)
  
  lld_rbf[ii] <- gpLogLikelihood(model0) ### loglikelihood ratio for rbf model
  
  model1 <- list() ## Allocate space for model.
  options=gpOptions(approx="ftc")
  options$kern = list(type="white") ### use white noise kernel for model 1
  ## Optimise GP log likelihoods.
  model1 <- gpCreate(dim(x)[2], dim(y)[2], x, y, options)
  model1 <- gpOptimise(model1,0)
  lld_static[ii] <- gpLogLikelihood(model1) ### loglikelihood ratio for static model
}




### an example for the 4th data point

### Get Loglikelihood ratio results: LR = -2 ln( L<sub>RBF</sub> - L<sub>static</sub>)

In [20]:
LR_data <- -2*(lld_rbf-lld_static) ### loglikelihood ratio for the data 


### Get BIC results: BIC = k ln(n) -2ln(L)
k is the number of parameters used in each model, n is the sample size and L is the maximized likelihood 

In [21]:
### BIC_difference is used to compare the BIC difference between RBF model and the static model. Smaller BIC is preferred
BIC_rbf <- 2*log(6) - 2*lld_rbf
BIC_static <- log(6) - 2*lld_static
BIC_difference <- BIC_rbf - BIC_static


In [22]:
data$BIC <- BIC_difference
data$LR <- LR_data

In [23]:
write.table(file='gene_normalized_new_withLRandBIC.txt',data, quote=F, row.names=F)
write.table(file='gene_normalized_new_withLRandBIC.csv',data, quote=F, row.names=F,sep=',')
